<a href="https://colab.research.google.com/github/ChiragBangera/MLspark/blob/main/Section1%2C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1

In [ ]:
# prompt: install pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=ddcf8fcf57d7e1b127e1661cad20c54a7ce7a5a5be40fafcbe3b9c744f0c0a02
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
# spark session creation
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Pyspark').getOrCreate()
spark

## Imports

In [ ]:
# imports
from pyspark.sql import functions as f
from pyspark.sql.types import *

## start

In [ ]:
data = [['Chirag',100],['Bangera',120],['Ronaldo', 99]]

df = spark.createDataFrame(data, ['Name', 'Score'])
df.show()

+-------+-----+
|   Name|Score|
+-------+-----+
| Chirag|  100|
|Bangera|  120|
|Ronaldo|   99|
+-------+-----+



In [ ]:
df.toPandas()

,Name,Score
0,Chirag,100
1,Bangera,120
2,Ronaldo,99


In [ ]:
# count rows
df.count()

3

In [ ]:
path = '/content/drive/MyDrive/spark/datasets/students.csv'
df = spark.read.csv(path, header = True)
df.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

In [ ]:
# groupby function
df.groupBy(['gender']).agg(F.round(F.mean('math score'))).show()

+------+-------------------------+
|gender|round(avg(math score), 0)|
+------+-------------------------+
|female|                     64.0|
|  male|                     69.0|
+------+-------------------------+



In [ ]:
df.groupby(['gender']).agg(F.min('math score'),F.max('math score'),F.round(F.mean('math score'),0)).show()

+------+---------------+---------------+-------------------------+
|gender|min(math score)|max(math score)|round(avg(math score), 0)|
+------+---------------+---------------+-------------------------+
|female|              0|             99|                     64.0|
|  male|            100|             99|                     69.0|
+------+---------------+---------------+-------------------------+



In [ ]:
df.rdd.id()

62

## Read, Write and Validate Code

In [ ]:
# checkign cores
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
cores

1

In [ ]:
path1 = '/content/drive/MyDrive/spark/datasets/'
students = spark.read.csv(path1+'students.csv',inferSchema = True, header=True)
students.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

In [ ]:
# parquet file
parquet = spark.read.parquet(path1+'users1.parquet')
parquet.limit(4).toPandas()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 07:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 17:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-03 01:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-03 00:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,


In [ ]:
# pertitioned datasets
partitioned = spark.read.parquet(path1+'users*')
partitioned.show(4)

+-------------------+---+----------+---------+--------------------+------+--------------+----------------+---------+---------+---------+--------------------+--------+
|  registration_dttm| id|first_name|last_name|               email|gender|    ip_address|              cc|  country|birthdate|   salary|               title|comments|
+-------------------+---+----------+---------+--------------------+------+--------------+----------------+---------+---------+---------+--------------------+--------+
|2016-02-03 07:55:29|  1|    Amanda|   Jordan|    ajordan0@com.com|Female|   1.197.201.2|6759521864920116|Indonesia| 3/8/1971| 49756.53|    Internal Auditor|   1E+02|
|2016-02-03 17:04:03|  2|    Albert|  Freeman|     afreeman1@is.gd|  Male|218.111.175.34|                |   Canada|1/16/1968|150280.17|       Accountant IV|        |
|2016-02-03 01:09:31|  3|    Evelyn|   Morgan|emorgan2@altervis...|Female|  7.161.136.94|6767119071901597|   Russia| 2/1/1960|144972.51| Structural Engineer|        

In [ ]:
# read only 2 data sets
users1_2 = spark.read.parquet(path1+'users1.parquet',path1+'users1.parquet')
users1_2.limit(4).toPandas()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 07:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 17:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-03 01:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-03 00:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,


In [ ]:
# s3 bucket sample
bucket = "my_bucket"
key1  = "partition_test/Table1/CREATED_YEAR=2015/*"
key2  = "partition_test/Table1/CREATED_YEAR=2017/*"
key3  = "partition_test/Table1/CREATED_YEAR=2018/*"
test_df = spark.read.parquet("s3a://"+bucket+"/"+key1, "s3a://"+bucket+"/"+key2, "s3a://"+bucket+"/"+key3)
test_df.limit(4).toPandas()

## Validating Data

In [ ]:
students.printSchema()

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)



In [ ]:
students.columns

['gender',
 'race/ethnicity',
 'parental level of education',
 'lunch',
 'test preparation course',
 'math score',
 'reading score',
 'writing score']

In [ ]:
students.describe().show()

+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|summary|gender|race/ethnicity|parental level of education|       lunch|test preparation course|        math score|     reading score|    writing score|
+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|  count|  1000|          1000|                       1000|        1000|                   1000|              1000|              1000|             1000|
|   mean|  NULL|          NULL|                       NULL|        NULL|                   NULL|            66.089|            69.169|           68.054|
| stddev|  NULL|          NULL|                       NULL|        NULL|                   NULL|15.163080096009454|14.600191937252223|15.19565701086966|
|    min|female|       group A|         associate's degree|free/reduced|          

In [ ]:
students.select('math score', 'reading score').summary('count','min','max').show()

+-------+----------+-------------+
|summary|math score|reading score|
+-------+----------+-------------+
|  count|      1000|         1000|
|    min|         0|           17|
|    max|       100|          100|
+-------+----------+-------------+



In [ ]:
students.schema['math score'].dataType

IntegerType()

## How to specify data types

In [ ]:
data_schema = StructType([
    StructField('name', StringType(), True),
    StructField('email', IntegerType(), True),
    StructField('city',StringType(),True),
    StructField('mac',StringType(),True),
    StructField('timestamp',DateType(),True),
    StructField('creditcard',StringType(),True)
])

In [ ]:
# people = spark.read.json(path1+'people.json',schema=data_schema)
# people.show()

## Writing data

In [ ]:
students.write.mode('overwrite').csv('write_test.csv')

In [ ]:
students.coalesce(1).write.mode('overwrite').csv('write_test_single_file.csv')

In [ ]:
users1_2.coalesce(2).write.mode('overwrite').parquet('write_test_parquet.parquet')

In [ ]:
fruit_values = [('Water Melon',100),('Orange',25),('Mango',100),('Pineapple',20)]

In [ ]:
fruits = spark.createDataFrame(fruit_values, schema=['name','qty'])
fruits.show()

+-----------+---+
|       name|qty|
+-----------+---+
|Water Melon|100|
|     Orange| 25|
|      Mango|100|
|  Pineapple| 20|
+-----------+---+



In [ ]:
@f.pandas_udf(IntegerType())
def add_one(val):
  return val + 20

adding_udf  = fruits.withColumn('qty_add',add_one('qty'))
adding_udf.show()

+-----------+---+-------+
|       name|qty|qty_add|
+-----------+---+-------+
|Water Melon|100|    120|
|     Orange| 25|     45|
|      Mango|100|    120|
|  Pineapple| 20|     40|
+-----------+---+-------+



In [ ]:
hunter